# **<span style="font-size:1em;">PrOMMiS LCA Integration: Environmental Life Cycle Assessment Using PrOMMiS Modeling Results</span>**

## Introduction

This document demonstrates the application of Life Cycle Assessment (LCA) methodology to the optimized results from the PrOMMiS model.  
This Jupyter Notebook presents an example LCA application to the West Kentucky No.13 Coal Refuse Flowsheet optimized in the PrOMMiS model.

**Useful notes, definitions, and links:**
- **PrOMMiS model:** Process Optimization and Modeling for Minerals Sustainability initiative, led by the U.S. Department of Energy (DOE), specifically under NETL (National Energy Technology Laboratory).
- **PrOMMiS public GitHub Repository:** [https://github.com/prommis/prommis/tree/main](https://github.com/prommis/prommis/tree/main)
- **UKy Flowsheet Code:** [https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py](https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py)

<div style="text-align: center;">
    <img src="images\uky_flowsheet.png" width="1000"/>
</div>

## Step 1: Import the necessary tools

In [ ]:
# import main libraries
import pandas as pd
import numpy as np
import olca_ipc
import olca_schema as olca
from netlolca.NetlOlca import NetlOlca
from typing import Union, List, Optional

from pyomo.environ import (units,value,)

import pubchempy as pcp

from pymatgen.core import Composition

from warnings import warn

# module to run the PrOMMiS model
import prommis.uky.uky_flowsheet as uky
# module to extract the data from the PrOMMiS model
from src import prommis_LCA_data
# module to run convert the PrOMMiS model data to LCA data
from src import prommis_LCA_conversions
# module to finalize the LCA data
from src import finalize_LCA_flows

# modules to connect to openLCA
from src.create_olca_process.olca_utils import (
    initialize_client, print_database_summary, generate_id, find_entity_by_name
)
from src.create_olca_process.dataframe_utils import (
    validate_dataframe_columns, load_dataframe_from_csv, get_amount_from_dataframe_row
)
from src.create_olca_process.search_utils import (
    get_existing_processes, search_processes_by_keywords
)
from src.create_olca_process.user_interface import (
    show_process_selection_menu, get_yes_no_input, get_user_search_choice, 
    show_unit_selection_in_flow_menu, show_flow_process_selection_menu
)
from src.create_olca_process.process_manager import create_process_from_dataframe_with_selection
from src.create_olca_process.flow_manager import find_compatible_flow_pattern, search_flows_by_name
from src.create_olca_process.unit_manager import (
    get_existing_units, get_common_unit_suggestions, search_units_by_name, 
    find_units_in_flow, find_unit_in_flow_by_name, create_unit_from_dataframe_row
)

## Step 2: Run PrOMMiS Optimization Model for the UKy flowsheet and extract model results 

In [ ]:
# Run PrOMMiS model 
m, results = uky.main()

# Extract LCA data
prommis_data = prommis_LCA_data.get_lca_df(m, )
prommis_data.to_csv("lca_df.csv")

## Step 3: Review PrOMMiS Results

In [ ]:
prommis_data.head()

## Step 4: Organize, Categorize, and Covert PrOMMiS Results to LCA Relevant Flows

In [ ]:
# Hours is the time period for the conversion
# mol_to_kg is a boolean that indicates whether to convert moles to kg
# water_unit is the unit of water (m3 or L recommended, kg is also an option)
df = prommis_LCA_conversions.convert_flows_to_lca_units(prommis_data, hours=1, mol_to_kg=True, water_unit='m3')
df.to_csv('lca_df_converted.csv', index=False)

# Finalize the LCA data
df = pd.read_csv('lca_df_converted.csv')
df = finalize_LCA_flows.main()
df.to_csv('lca_df_finalized.csv', index=False)

# This will create a new datafrane and csv file that contains the initial prommis_data df with two additional columns:
# 'LCA Amount' and 'LCA Unit'

## Step 6: Normalize Flows to the Selected Functional Unit and Review Final LCA Flows

In [ ]:
# The below code will normalize the converted prommis data to a functional unit
# Note: This code is developed for the UKy flowsheet and the functional unit is automatically set to 1 kg of REO (combination of all REEs)
# TODO: This code should be developed for the other flowsheets. To achieve this, the user should be able to specify the desired functional unit
df = pd.read_csv('lca_df_converted.csv')

# Run the merge_flows function for the feed
REO_list = [
    "Yttrium Oxide",
    "Lanthanum Oxide",
    "Cerium Oxide",
    "Praseodymium Oxide",
    "Neodymium Oxide",
    "Samarium Oxide",
    "Gadolinium Oxide",
    "Dysprosium Oxide",
]
df = finalize_LCA_flows.merge_flows(df, merge_source='Solid Feed', new_flow_name='374 ppm REO Feed', value_2_merge=REO_list)
# This 374 ppm value is directly calculated from the flowsheet. The original study actually used 357 ppm as the feed concentration.

# Run the merge_flows function for the product
df = finalize_LCA_flows.merge_flows(df, merge_source='Roaster Product', new_flow_name='99.85% REO Product')

# Run the merge_flows function for the liquid waste flows
df = finalize_LCA_flows.merge_flows(df, merge_source='Wastewater', new_flow_name='Wastewater', merge_column='Category') 
# Note: some of these streams are organic waste, but they're treated as wastewater

# Run the merge_flows function for the solid waste flows
df = finalize_LCA_flows.merge_flows(df, merge_source='Solid Waste', new_flow_name='Solid Waste', merge_column='Category') 

# Run the finalize_df function
try:
    finalized_df = finalize_LCA_flows.finalize_df(
        df=df, 
        reference_flow='99.85% REO Product', 
        reference_source='Roaster Product'
    )
    
    # Get summary
    summary = finalize_LCA_flows.get_finalize_summary(finalized_df)
    print("Summary:")
    for key, value in summary.items():
        if key != 'flow_type_breakdown':
            print(f"  {key}: {value}")
    
    print("\nFlow Type Breakdown:")
    for flow_type, count in summary['flow_type_breakdown'].items():
        print(f"  {flow_type}: {count}")
        
except Exception as e:
    print(f"Error during finalization: {e}")

finalized_df.to_csv('lca_df_finalized.csv', index=False)

## Step 7: Connect to openLCA

### Notes on modeling this process using openLCA

Before running the following code:
- Open the software openLCA
- Open an existing database or create a new one
- Import libraries with existing processes
- Use the IPC server from the Developer Tools
- Run the server on port 8080 

In [ ]:
print("\n🔌 STEP 1: Connecting to openLCA...")
client = initialize_client()

#### Define dataframe to be used

The dataframe should have the following columns: 

| Flow_Name | LCA_Unit | LCA_Amount | Is_Input | Reference_Product | Flow_Type | Description |

In [ ]:
# Save the LCA data produced in previous steps in a dataframe called df
df = pd.read_csv('lca_df_finalized.csv')


#### Enter Process Information
##### Unit Process Name

In [ ]:
process_name = "Test Process 1"  # Replace with the actual process name you want to use

#### Unit Process Description 

In [ ]:
process_description = " Example Process Description"  # Replace with the actual process description you want to use

#### Enter Unit Process Flows

In [ ]:
process = create_process_from_dataframe_with_selection(
    client=client,
    df=df,
    process_name=process_name,
    process_description=process_description
)


## Step 8: Create Process in openLCA from the final dataframe produced in Step 6 